In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [2]:
df = pd.read_excel('data/gb_sem_9_hw.xlsx')


ValueError: Excel file format cannot be determined, you must specify an engine manually.